In [42]:
import gensim
from gensim import corpora
from gensim import models, similarities
import os
import collections
import smart_open
import random

import pandas as pd
import numpy as np


from pprint import pprint
import string

# TODO
- Remove punctuations
- Filtering based on word count

In [16]:
def get_train_data():
    """
    SQL here 
    """
    data = pd.read_csv('~/Desktop/grocery_products.csv')
    return data

In [17]:
data = get_train_data()

In [18]:
print(data.shape)

(893626, 17)


In [19]:
selected_data = data[['source_product_id', 'name', 'departments_name', 'categories_name', 'description_en']]
# selected_data = selected_data.head(1000)

# # replace nan values with ''
selected_data = selected_data.replace(np.nan, '', regex=True)
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893626 entries, 0 to 893625
Data columns (total 5 columns):
source_product_id    893626 non-null int64
name                 893626 non-null object
departments_name     893626 non-null object
categories_name      893626 non-null object
description_en       893626 non-null object
dtypes: int64(1), object(4)
memory usage: 34.1+ MB


Creating document string

In [20]:
selected_data['document'] = selected_data['name'] + ' ' + selected_data['departments_name'] + ' ' + selected_data['categories_name'] + ' ' + selected_data['description_en']

In [21]:
selected_data[selected_data['source_product_id'] == 4979224]

,source_product_id,name,departments_name,categories_name,description_en,document
763,4979224,Tea Infusions - Ginger Tea,Drinks,Tea,Super uses sophisticated technology in their t...,Tea Infusions - Ginger Tea Drinks Tea Super us...


In [22]:
docs = selected_data['document'].tolist()
product_ids = selected_data['source_product_id'].tolist()

# Mapping from document index 
doc_index_to_id = {}
for index, product_id in enumerate(product_ids):
    doc_index_to_id[index] = product_id

In [23]:
doc_index_to_id[763]

4979224

In [24]:
pprint(docs[2])
# print(type(docs[0]))

('Ice Lemon Tea Drinks Tea Refreshing combination of fruity flavours and real '
 'tea leaves. Sweet cooling tea taste with a stimulating lemon flavour finish')


In [25]:
# pprint(docs)

In [26]:
def clean_documents(docs):
    """
    docs:
        list of string documents
    
    """
    # Remove punctuations
    translator = str.maketrans('', '', string.punctuation)
    docs = [d.translate(translator) for d in docs if isinstance(d, str)]
    
    # remove common words and tokenize
    stoplist = set('for a of the and to in'.split())
    docs = [[word for word in document.lower().split() if word not in stoplist]
             for document in docs]

    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in docs:
        for token in text:
            # TODO : Right now increasing by 2 to not skip any word
            frequency[token] += 2

    docs = [[token for token in doc if frequency[token] > 1] for doc in docs]

#     pprint(docs)
    return docs

In [27]:
docs = clean_documents(docs)

In [34]:
dictionary = corpora.Dictionary(docs)
dictionary.save(os.path.join('output_data', 'products_corpus.dict'))

In [ ]:
# print(dictionary)

In [ ]:
# print(dictionary.token2id)

In [35]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
corpora.MmCorpus.serialize(os.path.join('output_data', 'products_corpus.mm'), corpus)

In [36]:
! ls 'output_data'

deerwester.dict          products_corpus.mm
products_corpus.dict     products_corpus.mm.index


In [37]:
DATA_FOLDER = 'output_data'
dictionary = corpora.Dictionary.load(os.path.join(DATA_FOLDER, 'products_corpus.dict'))
corpus = corpora.MmCorpus(os.path.join(DATA_FOLDER, 'products_corpus.mm'))

In [38]:
print(corpus)

MmCorpus(893626 documents, 88913 features, 9835174 non-zero entries)


In [52]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=200)

In [53]:
index = similarities.MatrixSimilarity(lsi[corpus])

/Users/vipul/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Testing a sample document for similarity

In [103]:
sample_doc = 'green tea'

In [104]:
vec_bow = dictionary.doc2bow(sample_doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 0.06840769480056523), (1, 0.0735130106121753), (2, -0.06285905454710147), (3, 0.005913976527027322), (4, -0.02043975651246738), (5, -0.1841111541890951), (6, -0.09330512627513461), (7, 0.1609325334022964), (8, -0.2946336258745317), (9, -0.15990566745028928), (10, 0.045050276822673214), (11, -0.14544478823370344), (12, -0.03621941778137009), (13, -0.17492003995406438), (14, 0.35590843806218647), (15, -0.24753314023807943), (16, -0.661496768163064), (17, 0.5309233861647971), (18, -0.30740849351912564), (19, -0.14757877717942966), (20, -0.2635916180782732), (21, -0.07100966011404464), (22, 0.05495168093023563), (23, -0.021877022266308287), (24, 0.01052999308832735), (25, -0.016378749822509708), (26, 0.07922493731695456), (27, 0.005429530718993205), (28, -0.006512544451429965), (29, 0.0018543808337589795), (30, 0.010204425605846466), (31, -0.052870043110144815), (32, -0.013311693418198552), (33, -0.04592096043443648), (34, 0.04676906728110339), (35, -0.045085043574691834), (36, -0.022

In [105]:
similarity_scores = index[vec_lsi]
similarity_scores = list(enumerate(similarity_scores))

In [106]:
# Sort in descending order of similarity
sorted_similarity_scores = sorted(similarity_scores, key=lambda tup: tup[1], reverse=True)

In [107]:
# print(sorted_similarity_scores[:10])

In [108]:
type(sorted_similarity_scores[0])

tuple

In [109]:
for doc_index, score in sorted_similarity_scores[:10]:
    product_id = doc_index_to_id[doc_index]
    product_info = str(selected_data[selected_data['source_product_id'] == product_id]['name'])
    print(f'{product_info} : {score}')

785095    Green Jasmine
Name: name, dtype: object : 0.9997177124023438
815332    Green Jasmine
Name: name, dtype: object : 0.9997177124023438
399307    Oi Ocha Green Tea (tetra)
Name: name, dtype: object : 0.9859806299209595
98420    Green Pot Osmanthus Green Rooibos Tea
Name: name, dtype: object : 0.9666367173194885
552960    Makaibari Silver Green Darjeeling Green Tea
Name: name, dtype: object : 0.9663047790527344
654220    Green Pot Lemongrass Green Tea
Name: name, dtype: object : 0.9659451246261597
721851    Oi Ocha Green Tea
Name: name, dtype: object : 0.9493176937103271
449182    Green tea leaves
Name: name, dtype: object : 0.9451427459716797
94444    Silver Green Darjeeling Green Tea
Name: name, dtype: object : 0.9445038437843323
150654    Silver Green Darjeeling Green Tea
Name: name, dtype: object : 0.9445038437843323
